In [2]:
from schrodinger import Schrodinger2D, Rectangle

s = Schrodinger2D(lambda x, y: x**2 + y**2, Rectangle(-9.5,9.5, -9.5,9.5),
                  gridSize=(64,64), maxBasisSize=32, sparse=True)
s.eigenvalues(21)

[2.0000000000000075,
 4.000000000000001,
 4.0000000000000195,
 5.999999999999976,
 5.999999999999976,
 6.0000000000000275,
 7.999999999999972,
 7.999999999999972,
 7.999999999999979,
 8.000000000000073,
 9.999999999999952,
 9.999999999999993,
 10.00000000000001,
 10.000000000000014,
 10.000000000000053,
 11.999999999999861,
 11.999999999999995,
 11.999999999999995,
 12.000000000000025,
 12.000000000000151,
 13.999999999999892]

[1.999998922660854,
 4.0000236152640865,
 4.000023615264108,
 5.9997813728226035,
 5.999781375136523,
 6.000048309300554,
 7.999806031607065,
 7.999806031607065,
 8.003147793568461,
 8.003147793568461]